# Mapa de calor por ciudades

Se genera un mapa de calor sobre un mapa de méxico, indicando zonas de calor en las ciudades de los participantes, mientras más jugadores provengan de una ciudad, más cálido será el color

In [1]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import time
import requests

In [2]:
# Datos de ejemplo
data = {
    "ciudad": ["Salvatierra", "Abasolo", "Morelia", "Irapuato", "Ciudad de México"],
    "jugadores": [5, 10, 4, 20, 2]  # Número de jugadores por ciudad
}
df = pd.DataFrame(data)

In [3]:
df

,ciudad,jugadores
0,Salvatierra,5
1,Abasolo,10
2,Morelia,4
3,Irapuato,20
4,Ciudad de México,2


In [4]:
# Usar API de Nominatim para obtener coordenadas
def get_coords_nominatim(ciudad):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"q": f"{ciudad}, México", "format": "json", "limit": 1}
    headers = {"User-Agent": "torneo_gobierno_irapuato_frontenis_v1.0 (contacto@irapuato.gob.mx)"}

    try:
        response = requests.get(url, params=params, headers=headers, timeout=15)
        response.raise_for_status()  # Lanza error para códigos 4xx/5xx
        data = response.json()
        return [float(data[0]["lat"]), float(data[0]["lon"])] if data else None
    except requests.exceptions.RequestException as e:
        print(f"Error en {ciudad}: {str(e)}")
        return None
# Aplicar
df["coordenadas"] = df["ciudad"].apply(
    lambda x: (time.sleep(1), get_coords_nominatim(x))[1]  # Pausa de 1s entre solicitudes
)

In [5]:
df

,ciudad,jugadores,coordenadas
0,Salvatierra,5,"[20.192654599999997, -100.86599313440021]"
1,Abasolo,10,"[20.5375581, -101.54578687631809]"
2,Morelia,4,"[19.6546004, -101.26237972291207]"
3,Irapuato,20,"[20.6758761, -101.3521052]"
4,Ciudad de México,2,"[19.4326296, -99.1331785]"


In [8]:
#Crear un mapa con folium y agregar areas de calor según el numero de jugadores
mapa = folium.Map(
    location=[20.6736, -101.325],  # Centro cerca de Irapuato para mejor enfoque
    zoom_start=7,
    tiles="CartoDB dark_matter", # Mapa de aspecto oscuro, creo que esto hará que se mezcle mejor con el diseño de la pagina
    attr='© CARTO')

#Preparar datos para el heatmap (coordenadas + peso)
heat_data = [[row["coordenadas"][0], row["coordenadas"][1], row["jugadores"]]
             for _, row in df.iterrows()]

HeatMap(heat_data, radius=25, blur=15).add_to(mapa)

In [9]:
mapa